In [1]:
import pandas as pd
import numpy as np

In [3]:
all_data = pd.read_csv('iris\iris.csv')

In [4]:
all_data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
all_data['species'] = label_encoder.fit_transform(all_data['species'])
all_data['species'].unique()

all_data = all_data[all_data['species'] != 2]
all_data['species'].unique()


array([0, 1], dtype=int64)

In [7]:
from sklearn.model_selection import train_test_split

all_labels = all_data['species'].copy()
all_features = all_data.drop(['species'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.3, random_state=0)

In [8]:
X_train.shape, y_train.shape

((70, 4), (70,))

In [9]:
X_test.shape, y_test.shape

((30, 4), (30,))

In [10]:
# Reshape features for NN
X_train = X_train.T
X_test = X_test.T

In [11]:
# Reshape labels for NN
y_train = (pd.DataFrame(y_train)).T
y_test = (pd.DataFrame(y_test)).T

In [12]:
# Get the node sizes for NN
n_x = X_train.shape[0]
n_h = 6
n_y = y_train.shape[0]

### Initialize the parameters

In [13]:
np.random.seed(2)

W1 = np.random.randn(n_h, n_x) * 0.01
b1 = np.random.randn(n_h, 1) * 0.01
W2 = np.random.randn(n_y, n_h) * 0.01
b2 = np.random.randn(n_y, 1) * 0.01

In [14]:
def tanh(Z):
    return np.tanh(Z)

In [15]:
def sigmoid(Z):
    return (1 / (1 + np.exp(-Z)))

In [16]:
def derivative_tanh(Z, A1):
    return (1 - np.power(A1, 2))

### Forward-Propagate

In [17]:
def forward_propagate(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X) + b1
    A1 = tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {
        'Z1': Z1,
        'A1': A1,
        'Z2': Z2,
        'A2': A2
    }
    
    return cache

In [18]:
forward_try_result = forward_propagate(X_train, W1, b1, W2, b2)
forward_try_result['A2'].shape

(1, 70)

### Cost Function

In [19]:
def compute_cost(A2, Y, W1, b1, W2, b2):
    m = Y.shape[1]
    
    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1-Y), np.log(1-A2))
    
    cost = - np.sum(logprobs, axis=1) / m
    cost = cost.to_numpy().reshape(-1, 1)
    
    return cost

In [20]:
cost_try_result = compute_cost(forward_try_result['A2'], y_train, W1, b1, W2, b2)
cost_try_result.shape

(1, 1)

### Backward-Propagate

In [21]:
def backward_propagagate(X, Y, forward_result, W1, b1, W2, b2):
    m = X.shape[1]
    
    A1 = forward_result['A1']
    A2 = forward_result['A2']
    Z1 = forward_result['Z1']
    
    dZ2 = A2 - Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1).to_numpy().reshape(-1, 1)
    
    dZ1 = np.dot(W2.T, dZ2) * derivative_tanh(Z1, A1)
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1).reshape(-1, 1)
    
    grads = {
        'dZ2': dZ2,
        'dW2': dW2,
        'db2': db2,
        'dZ1': dZ1,
        'dW1': dW1,
        'db1': db1
    }
#     return None
    return grads

In [22]:
backward_try_result = backward_propagagate(X_train, y_train, forward_try_result, W1, b1, W2, b2)
backward_try_result['db1'].shape

(6, 1)

In [23]:
def update_params(W1, b1, W2, b2, backward_result, learning_rate=0.001):
    
    dW1 = backward_result['dW1']
    db1 = backward_result['db1']
    dW2 = backward_result['dW2']
    db2 = backward_result['db2']
    
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    
    return W1, b1, W2, b2

In [24]:
# Train the Neural Network!
def nn_model(X, Y, W1, b1, W2, b2, num_iterations=1000):
    for i in range(0, num_iterations):
        forward_result = forward_propagate(X, W1, b1, W2, b2)
        cost = compute_cost(forward_result['A2'], Y, W1, b1, W2, b2)
        backward_result = backward_propagagate(X, Y, forward_result, W1, b1, W2, b2)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, backward_result, learning_rate=0.009)
        
        print('Cost after iteration i:', i, cost) 
        
    return W1, b1, W2, b2

In [25]:
W1, b1, W2, b2 = nn_model(X_train, y_train, W1, b1, W2, b2, num_iterations=1000)

Cost after iteration i: 0 [[0.69345491]]
Cost after iteration i: 1 [[0.69344509]]
Cost after iteration i: 2 [[0.69343533]]
Cost after iteration i: 3 [[0.69342564]]
Cost after iteration i: 4 [[0.69341599]]
Cost after iteration i: 5 [[0.6934064]]
Cost after iteration i: 6 [[0.69339687]]
Cost after iteration i: 7 [[0.69338738]]
Cost after iteration i: 8 [[0.69337795]]
Cost after iteration i: 9 [[0.69336856]]
Cost after iteration i: 10 [[0.69335922]]
Cost after iteration i: 11 [[0.69334992]]
Cost after iteration i: 12 [[0.69334066]]
Cost after iteration i: 13 [[0.69333144]]
Cost after iteration i: 14 [[0.69332225]]
Cost after iteration i: 15 [[0.69331311]]
Cost after iteration i: 16 [[0.693304]]
Cost after iteration i: 17 [[0.69329492]]
Cost after iteration i: 18 [[0.69328587]]
Cost after iteration i: 19 [[0.69327684]]
Cost after iteration i: 20 [[0.69326785]]
Cost after iteration i: 21 [[0.69325888]]
Cost after iteration i: 22 [[0.69324993]]
Cost after iteration i: 23 [[0.69324101]]
Cost 

Cost after iteration i: 194 [[0.69025558]]
Cost after iteration i: 195 [[0.6902151]]
Cost after iteration i: 196 [[0.69017411]]
Cost after iteration i: 197 [[0.69013259]]
Cost after iteration i: 198 [[0.69009054]]
Cost after iteration i: 199 [[0.69004796]]
Cost after iteration i: 200 [[0.69000483]]
Cost after iteration i: 201 [[0.68996116]]
Cost after iteration i: 202 [[0.68991694]]
Cost after iteration i: 203 [[0.68987216]]
Cost after iteration i: 204 [[0.6898268]]
Cost after iteration i: 205 [[0.68978088]]
Cost after iteration i: 206 [[0.68973437]]
Cost after iteration i: 207 [[0.68968728]]
Cost after iteration i: 208 [[0.68963959]]
Cost after iteration i: 209 [[0.68959131]]
Cost after iteration i: 210 [[0.68954241]]
Cost after iteration i: 211 [[0.6894929]]
Cost after iteration i: 212 [[0.68944277]]
Cost after iteration i: 213 [[0.68939201]]
Cost after iteration i: 214 [[0.68934062]]
Cost after iteration i: 215 [[0.68928858]]
Cost after iteration i: 216 [[0.68923589]]
Cost after ite

Cost after iteration i: 386 [[0.66254949]]
Cost after iteration i: 387 [[0.6622195]]
Cost after iteration i: 388 [[0.66188632]]
Cost after iteration i: 389 [[0.6615499]]
Cost after iteration i: 390 [[0.66121022]]
Cost after iteration i: 391 [[0.66086724]]
Cost after iteration i: 392 [[0.66052093]]
Cost after iteration i: 393 [[0.66017128]]
Cost after iteration i: 394 [[0.65981823]]
Cost after iteration i: 395 [[0.65946177]]
Cost after iteration i: 396 [[0.65910186]]
Cost after iteration i: 397 [[0.65873846]]
Cost after iteration i: 398 [[0.65837156]]
Cost after iteration i: 399 [[0.65800111]]
Cost after iteration i: 400 [[0.65762709]]
Cost after iteration i: 401 [[0.65724945]]
Cost after iteration i: 402 [[0.65686818]]
Cost after iteration i: 403 [[0.65648323]]
Cost after iteration i: 404 [[0.65609458]]
Cost after iteration i: 405 [[0.65570219]]
Cost after iteration i: 406 [[0.65530602]]
Cost after iteration i: 407 [[0.65490605]]
Cost after iteration i: 408 [[0.65450224]]
Cost after it

Cost after iteration i: 580 [[0.50557034]]
Cost after iteration i: 581 [[0.50422101]]
Cost after iteration i: 582 [[0.50286775]]
Cost after iteration i: 583 [[0.50151062]]
Cost after iteration i: 584 [[0.5001497]]
Cost after iteration i: 585 [[0.49878505]]
Cost after iteration i: 586 [[0.49741674]]
Cost after iteration i: 587 [[0.49604483]]
Cost after iteration i: 588 [[0.49466939]]
Cost after iteration i: 589 [[0.49329049]]
Cost after iteration i: 590 [[0.49190821]]
Cost after iteration i: 591 [[0.4905226]]
Cost after iteration i: 592 [[0.48913375]]
Cost after iteration i: 593 [[0.48774171]]
Cost after iteration i: 594 [[0.48634657]]
Cost after iteration i: 595 [[0.4849484]]
Cost after iteration i: 596 [[0.48354726]]
Cost after iteration i: 597 [[0.48214323]]
Cost after iteration i: 598 [[0.48073638]]
Cost after iteration i: 599 [[0.47932679]]
Cost after iteration i: 600 [[0.47791453]]
Cost after iteration i: 601 [[0.47649968]]
Cost after iteration i: 602 [[0.4750823]]
Cost after iter

Cost after iteration i: 774 [[0.24999387]]
Cost after iteration i: 775 [[0.24900269]]
Cost after iteration i: 776 [[0.24801603]]
Cost after iteration i: 777 [[0.24703387]]
Cost after iteration i: 778 [[0.24605622]]
Cost after iteration i: 779 [[0.24508306]]
Cost after iteration i: 780 [[0.24411438]]
Cost after iteration i: 781 [[0.24315018]]
Cost after iteration i: 782 [[0.24219044]]
Cost after iteration i: 783 [[0.24123516]]
Cost after iteration i: 784 [[0.24028433]]
Cost after iteration i: 785 [[0.23933793]]
Cost after iteration i: 786 [[0.23839596]]
Cost after iteration i: 787 [[0.23745841]]
Cost after iteration i: 788 [[0.23652527]]
Cost after iteration i: 789 [[0.23559652]]
Cost after iteration i: 790 [[0.23467216]]
Cost after iteration i: 791 [[0.23375217]]
Cost after iteration i: 792 [[0.23283654]]
Cost after iteration i: 793 [[0.23192527]]
Cost after iteration i: 794 [[0.23101833]]
Cost after iteration i: 795 [[0.23011572]]
Cost after iteration i: 796 [[0.22921742]]
Cost after 

Cost after iteration i: 967 [[0.12611115]]
Cost after iteration i: 968 [[0.12572902]]
Cost after iteration i: 969 [[0.12534871]]
Cost after iteration i: 970 [[0.1249702]]
Cost after iteration i: 971 [[0.12459348]]
Cost after iteration i: 972 [[0.12421855]]
Cost after iteration i: 973 [[0.12384538]]
Cost after iteration i: 974 [[0.12347399]]
Cost after iteration i: 975 [[0.12310434]]
Cost after iteration i: 976 [[0.12273643]]
Cost after iteration i: 977 [[0.12237026]]
Cost after iteration i: 978 [[0.12200581]]
Cost after iteration i: 979 [[0.12164308]]
Cost after iteration i: 980 [[0.12128204]]
Cost after iteration i: 981 [[0.1209227]]
Cost after iteration i: 982 [[0.12056504]]
Cost after iteration i: 983 [[0.12020906]]
Cost after iteration i: 984 [[0.11985474]]
Cost after iteration i: 985 [[0.11950207]]
Cost after iteration i: 986 [[0.11915105]]
Cost after iteration i: 987 [[0.11880166]]
Cost after iteration i: 988 [[0.1184539]]
Cost after iteration i: 989 [[0.11810776]]
Cost after ite

In [26]:
prediction_results = forward_propagate(X_train, W1, b1, W2, b2)

predictions = prediction_results['A2']

for pred in range(predictions.shape[1]):
    if predictions[:, pred] <= 0.5:
        predictions[:, pred] = 0
    else:
        predictions[:, pred] = 1

train_pred = pd.DataFrame({'prediction': predictions.squeeze(),
              'actual': y_train.squeeze()
             })
print(train_pred.head(15))

    prediction  actual
60         1.0       1
80         1.0       1
90         1.0       1
68         1.0       1
51         1.0       1
27         0.0       0
18         0.0       0
56         1.0       1
63         1.0       1
74         1.0       1
1          0.0       0
61         1.0       1
42         0.0       0
41         0.0       0
4          0.0       0


In [27]:
ave_err = np.sum(train_pred['prediction'] - train_pred['actual']) / len(train_pred)
ave_err

0.0

### Test

In [28]:
test_prediction_results = forward_propagate(X_test, W1, b1, W2, b2)

test_predictions = test_prediction_results['A2']

for pred in range(test_predictions.shape[1]):
    if test_predictions[:, pred] <= 0.5:
        test_predictions[:, pred] = 0
    else:
        test_predictions[:, pred] = 1

test_pred = pd.DataFrame({'prediction': test_predictions.squeeze(),
              'actual': y_test.squeeze()
             })

test_pred.head()

,prediction,actual
26,0.0,0
86,1.0,1
2,0.0,0
55,1.0,1
75,1.0,1


In [29]:

ave_err = np.sum(test_pred['prediction'] - test_pred['actual']) / len(test_pred)
print('Average error: ', ave_err)

Average error:  0.0
